![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [1]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, GridSearchCV

from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


# URLs dos arquivos
filename_features = "https://raw.githubusercontent.com/Jefersonfranca/EBAC_Curso_Cientista_de_Dados/main/M%C3%B3dulo%2017%20%C3%81rvores%20II/dataset/features_info.txt"
filename_labels = "https://raw.githubusercontent.com/Jefersonfranca/EBAC_Curso_Cientista_de_Dados/main/M%C3%B3dulo%2017%20%C3%81rvores%20II/dataset/activity_labels.txt"
filename_subtrain = "https://raw.githubusercontent.com/Jefersonfranca/EBAC_Curso_Cientista_de_Dados/main/M%C3%B3dulo%2017%20%C3%81rvores%20II/dataset/subject_train.txt"
filename_xtrain = "https://raw.githubusercontent.com/Jefersonfranca/EBAC_Curso_Cientista_de_Dados/main/M%C3%B3dulo%2017%20%C3%81rvores%20II/dataset/X_train.txt"
filename_ytrain = "https://raw.githubusercontent.com/Jefersonfranca/EBAC_Curso_Cientista_de_Dados/main/M%C3%B3dulo%2017%20%C3%81rvores%20II/dataset/y_train.txt"
filename_subtest = "https://raw.githubusercontent.com/Jefersonfranca/EBAC_Curso_Cientista_de_Dados/main/M%C3%B3dulo%2017%20%C3%81rvores%20II/dataset/subject_test.txt"
filename_xtest = "https://raw.githubusercontent.com/Jefersonfranca/EBAC_Curso_Cientista_de_Dados/main/M%C3%B3dulo%2017%20%C3%81rvores%20II/dataset/X_test.txt"
filename_ytest = "https://raw.githubusercontent.com/Jefersonfranca/EBAC_Curso_Cientista_de_Dados/main/M%C3%B3dulo%2017%20%C3%81rvores%20II/dataset/y_test.txt"

# Carregar nomes de features
features = pd.read_csv(filename_features, header=None, sep="#")

# Definir os nomes das colunas
features.columns = ['nome_var']

# Carregar labels
labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])

# Carregar dados de treino
subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'])
X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None, names=features['nome_var'].tolist())
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

# Carregar dados de teste
subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'])
X_test = pd.read_csv(filename_xtest, delim_whitespace=True, header=None, names=features['nome_var'].tolist())
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])


In [2]:
# Dividindo a base em Treino, Validação e Teste
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train)

## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [3]:
%%time
ccp_alpha = 0.001
clf = DecisionTreeClassifier(ccp_alpha=ccp_alpha, random_state=42).fit(X_train, y_train)
#clf = DecisionTreeClassifier(random_state=1234)
caminho = DecisionTreeClassifier(random_state=2360873, min_samples_leaf=20).cost_complexity_pruning_path(X_train, y_train)
ccp_alphas, impurities = caminho.ccp_alphas, caminho.impurities

ccp_alphas = np.unique(ccp_alphas[ccp_alphas>=0])

clfs = []
for ccp_alpha in ccp_alphas:
    clf = DecisionTreeClassifier(random_state=2360873, ccp_alpha=ccp_alpha).fit(X_train, y_train)
    clfs.append(clf)

CPU times: total: 2.02 s
Wall time: 22.2 s


In [4]:
%%time
# Treinar o classificador
clf.fit(X_train, y_train)

# Avaliar a acurácia nos dados de treinamento
y_train_pred = clf.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)

# Avaliar a acurácia nos dados de teste
y_test_pred = clf.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)

# Exibir os resultados
print(f"Acurácia no conjunto de treinamento: {train_accuracy:.4f}")
print(f"Acurácia no conjunto de teste: {test_accuracy:.4f}")

Acurácia no conjunto de treinamento: 0.1904
Acurácia no conjunto de teste: 0.1822
CPU times: total: 31.2 ms
Wall time: 276 ms


## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [5]:
%%time

# Definir o número de componentes principais desejadas
n_components = 1

# Aplicar PCA para reduzir as variáveis
pca = PCA(n_components=n_components, random_state=42)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Criar o classificador de árvore de decisão
clf = DecisionTreeClassifier(random_state=42)

# Treinar o classificador
clf.fit(X_train_pca, y_train)

# Avaliar a acurácia nos dados de treinamento
y_train_pred = clf.predict(X_train_pca)
train_accuracy = accuracy_score(y_train, y_train_pred)

# Avaliar a acurácia nos dados de teste
y_test_pred = clf.predict(X_test_pca)
test_accuracy = accuracy_score(y_test, y_test_pred)

# Exibir os resultados
print(f"Acurácia no conjunto de treinamento: {train_accuracy:.4f}")
print(f"Acurácia no conjunto de teste: {test_accuracy:.4f}")


Acurácia no conjunto de treinamento: 0.9998
Acurácia no conjunto de teste: 0.4435
CPU times: total: 125 ms
Wall time: 53 ms


## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [6]:
# Lista de números de componentes principais a testar
componentes_principais = [1, 2, 5, 10, min(X_train.shape)]  # Adicionando min(X_train.shape) como máximo pois da outra vez com 50 havia dado erro

# Loop sobre os diferentes números de componentes
for n_components in componentes_principais:
    print(f"### Avaliando com {n_components} componentes principais ###")
    
    # Aplicar PCA para reduzir as variáveis
    pca = PCA(n_components=n_components, random_state=42)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)

    # Criar o classificador de árvore de decisão
    clf = DecisionTreeClassifier(random_state=42)

    # Treinar o classificador
    clf.fit(X_train_pca, y_train)

    # Avaliar a acurácia nos dados de treinamento
    y_train_pred = clf.predict(X_train_pca)
    train_accuracy = accuracy_score(y_train, y_train_pred)

    # Avaliar a acurácia nos dados de teste
    y_test_pred = clf.predict(X_test_pca)
    test_accuracy = accuracy_score(y_test, y_test_pred)

    # Exibir os resultados
    print(f"Acurácia no conjunto de treinamento: {train_accuracy:.4f}")
    print(f"Acurácia no conjunto de teste: {test_accuracy:.4f}")
    print()

### Avaliando com 1 componentes principais ###
Acurácia no conjunto de treinamento: 1.0000
Acurácia no conjunto de teste: 0.4143

### Avaliando com 2 componentes principais ###
Acurácia no conjunto de treinamento: 1.0000
Acurácia no conjunto de teste: 0.4459

### Avaliando com 5 componentes principais ###
Acurácia no conjunto de treinamento: 1.0000
Acurácia no conjunto de teste: 0.6685

### Avaliando com 10 componentes principais ###
Acurácia no conjunto de treinamento: 1.0000
Acurácia no conjunto de teste: 0.6946

### Avaliando com 49 componentes principais ###
Acurácia no conjunto de treinamento: 1.0000
Acurácia no conjunto de teste: 0.7207



In [6]:
%%time

print(f"### Avaliando com {1} componentes principais ###")

n_components = 1

# Aplicar PCA para reduzir as variáveis
pca = PCA(n_components=n_components, random_state=42)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Criar o classificador de árvore de decisão
clf = DecisionTreeClassifier(random_state=42)

# Treinar o classificador
clf.fit(X_train_pca, y_train)

# Avaliar a acurácia nos dados de treinamento
y_train_pred = clf.predict(X_train_pca)
train_accuracy = accuracy_score(y_train, y_train_pred)

# Avaliar a acurácia nos dados de teste
y_test_pred = clf.predict(X_test_pca)
test_accuracy = accuracy_score(y_test, y_test_pred)

# Exibir os resultados
print(f"Acurácia no conjunto de treinamento: {train_accuracy:.4f}")
print(f"Acurácia no conjunto de teste: {test_accuracy:.4f}")
print()

### Avaliando com 1 componentes principais ###
Acurácia no conjunto de treinamento: 0.9998
Acurácia no conjunto de teste: 0.4435

CPU times: total: 93.8 ms
Wall time: 29 ms


In [7]:
%%time

print(f"### Avaliando com {2} componentes principais ###")

n_components = 2
    
# Aplicar PCA para reduzir as variáveis
pca = PCA(n_components=n_components, random_state=42)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Criar o classificador de árvore de decisão
clf = DecisionTreeClassifier(random_state=42)

# Treinar o classificador
clf.fit(X_train_pca, y_train)

# Avaliar a acurácia nos dados de treinamento
y_train_pred = clf.predict(X_train_pca)
train_accuracy = accuracy_score(y_train, y_train_pred)

# Avaliar a acurácia nos dados de teste
y_test_pred = clf.predict(X_test_pca)
test_accuracy = accuracy_score(y_test, y_test_pred)

# Exibir os resultados
print(f"Acurácia no conjunto de treinamento: {train_accuracy:.4f}")
print(f"Acurácia no conjunto de teste: {test_accuracy:.4f}")
print()

### Avaliando com 2 componentes principais ###
Acurácia no conjunto de treinamento: 1.0000
Acurácia no conjunto de teste: 0.4666

CPU times: total: 125 ms
Wall time: 39.6 ms


In [8]:
%%time

print(f"### Avaliando com {5} componentes principais ###")

n_components = 5
    
# Aplicar PCA para reduzir as variáveis
pca = PCA(n_components=n_components, random_state=42)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Criar o classificador de árvore de decisão
clf = DecisionTreeClassifier(random_state=42)

# Treinar o classificador
clf.fit(X_train_pca, y_train)

# Avaliar a acurácia nos dados de treinamento
y_train_pred = clf.predict(X_train_pca)
train_accuracy = accuracy_score(y_train, y_train_pred)

# Avaliar a acurácia nos dados de teste
y_test_pred = clf.predict(X_test_pca)
test_accuracy = accuracy_score(y_test, y_test_pred)

# Exibir os resultados
print(f"Acurácia no conjunto de treinamento: {train_accuracy:.4f}")
print(f"Acurácia no conjunto de teste: {test_accuracy:.4f}")
print()

### Avaliando com 5 componentes principais ###
Acurácia no conjunto de treinamento: 1.0000
Acurácia no conjunto de teste: 0.6634

CPU times: total: 109 ms
Wall time: 53 ms


In [9]:
%%time

print(f"### Avaliando com {10} componentes principais ###")

n_components = 10
    
# Aplicar PCA para reduzir as variáveis
pca = PCA(n_components=n_components, random_state=42)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Criar o classificador de árvore de decisão
clf = DecisionTreeClassifier(random_state=42)

# Treinar o classificador
clf.fit(X_train_pca, y_train)

# Avaliar a acurácia nos dados de treinamento
y_train_pred = clf.predict(X_train_pca)
train_accuracy = accuracy_score(y_train, y_train_pred)

# Avaliar a acurácia nos dados de teste
y_test_pred = clf.predict(X_test_pca)
test_accuracy = accuracy_score(y_test, y_test_pred)

# Exibir os resultados
print(f"Acurácia no conjunto de treinamento: {train_accuracy:.4f}")
print(f"Acurácia no conjunto de teste: {test_accuracy:.4f}")
print()

### Avaliando com 10 componentes principais ###
Acurácia no conjunto de treinamento: 1.0000
Acurácia no conjunto de teste: 0.6912

CPU times: total: 422 ms
Wall time: 130 ms


In [11]:
%%time

print(f"### Avaliando com {50} componentes principais ###")

n_components = 49 # 50 ele não passa
    
# Aplicar PCA para reduzir as variáveis
pca = PCA(n_components=n_components, random_state=42)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Criar o classificador de árvore de decisão
clf = DecisionTreeClassifier(random_state=42)

# Treinar o classificador
clf.fit(X_train_pca, y_train)

# Avaliar a acurácia nos dados de treinamento
y_train_pred = clf.predict(X_train_pca)
train_accuracy = accuracy_score(y_train, y_train_pred)

# Avaliar a acurácia nos dados de teste
y_test_pred = clf.predict(X_test_pca)
test_accuracy = accuracy_score(y_test, y_test_pred)

# Exibir os resultados
print(f"Acurácia no conjunto de treinamento: {train_accuracy:.4f}")
print(f"Acurácia no conjunto de teste: {test_accuracy:.4f}")
print()

### Avaliando com 50 componentes principais ###
Acurácia no conjunto de treinamento: 1.0000
Acurácia no conjunto de teste: 0.6973

CPU times: total: 1.09 s
Wall time: 381 ms


## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

- **O que aconteceu com a acurácia?**
   A acurácia aumentou à medida que aumentei a quantidade de componentes principais. Isso indica que as representações dos dados foram melhoradas à medida que mais informações foram capturadas pelas componentes principais, resultando em um melhor desempenho preditivo do modelo.

- **O que aconteceu com o tempo de processamento?**
   O tempo de processamento também aumentou à medida que aumentei a quantidade de componentes principais. Isso ocorre porque o PCA, ao extrair mais componentes principais, requer mais cálculos computacionais para transformar os dados e treinar o modelo.